# Geolocaliser les évènements pour Rabat

In [1]:
import requests

In [2]:
url = 'https://www.ticket.ma/events/search?q=rabat'
response = requests.get(url)
response

<Response [200]>

In [3]:
print(response.text[:100])

<html xmlns="//www.w3.org/1999/xhtml">
	<head>
	    <meta property="fb:app_id" content="273870459404


In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text,'html.parser')
type(soup)

bs4.BeautifulSoup

In [5]:
page_html = soup.find('div',{"class":"home clearfix local-events"})

In [6]:
ul = page_html.find('ul',class_='clearfix')

In [7]:
ul

<ul class="clearfix">
<li class="tfly-event-id- vevent" id="event-2051">
<a class="data-link" href="/event/sqnc-le-03-aout-au-dhow-rabat">
<img src="/upload/events/2051/original_205_4.png"> </img></a>
<div class="activity-tools">
<p>
<a class="overlay-trigger" href="/event/sqnc-le-03-aout-au-dhow-rabat">Acheter</a>
</p>
<ul>
<li><a class="activity-tools-facebook" href="javascript:void(0);" onclick="showBShareDialog('event-2051');return false">Facebook</a></li>
<li><a class="activity-tools-twitter" href="https://twitter.com/intent/tweet?text=SQNC.&amp;url=https%3A%2F%2Fwww.ticket.ma%2Fevent%2Fsqnc-le-03-aout-au-dhow-rabat&amp;via=ticket_ma" target="_blank">Twitter</a></li>
</ul>
</div>
<h2 class="headliners summary">
<a class="overlay-trigger" href="/event/sqnc-le-03-aout-au-dhow-rabat">SQNC. </a>
</h2>
<p class="event-date">
				Le samedi 03 août 2019 · 200 Dhs			
			</p>
<p class="venue location location_indicator">
<a href="/event/sqnc-le-03-aout-au-dhow-rabat">Le Dhow - Rabat</a><br

In [8]:
#
events = ul.find_all('a',{"class":"data-link"})

### Scraping des liens

In [9]:
liens = []

In [10]:
for a in events:
    liens.append("https://www.ticket.ma"+str(a['href']))    

In [11]:
liens

['https://www.ticket.ma/event/sqnc-le-03-aout-au-dhow-rabat',
 'https://www.ticket.ma/event/convention-nature-4-ever-au-maroc-le-28-septembre-2019-rabat']

### Scraping des dates (format plus interessant sur la première page)


In [14]:
d = []
for i in page_html.find_all('p',{"class":"event-date"}):
    d.append(i.get_text().replace("\t","").replace("\n",""))

In [18]:
d

['Le samedi 03 août 2019 · 200 Dhs', 'Le samedi 28 septembre 2019 · 80 Dhs']

In [21]:
dates = []
for i in d:
    a,b = i.split('·')
    dates.append(a.replace("Le","").strip())

In [22]:
dates

['samedi 03 août 2019', 'samedi 28 septembre 2019']

## Récupérer les données lien par lien

In [25]:
from time import time
from time import sleep
from random import randint

In [26]:
start_time = time()
requests = 0

In [27]:
from requests import get

In [45]:
titres = []
descs = []
images = []
heures = []
places = []

In [48]:
for i in liens:
    print(i)
    response = get(i)
    
    timer = randint(8,15)
    #pause de loop
    print(timer)
    sleep(timer)
    
    #renvoyer un warning pour les non 'status code : 200'
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            
    #parser en BS
    page_html = BeautifulSoup(response.text,'html.parser')
    
    """ Titres """
    title = page_html.find('div',class_='event-titles')
    titres.append((title.text).strip())
    
    """ Descriptions """
    detail = page_html.find('div',class_='event-details')
    descs.append((detail.find('div').text).strip())
    
    """ Images """
    img = page_html.find(id="artist-image")
    image = img.find('img')['src']
    images.append("https://www.ticket.ma"+str(image))
    
    """ Heures """
    h = (page_html.find('span',class_='start dtstart')).text
    heures.append(h[-5:])#extraire les derniers elements de la liste
    
    """ Places """
    place = page_html.find('a',class_='fancybox map-location')
    places.append(place.text)
    

print("Done")
    

https://www.ticket.ma/event/sqnc-le-03-aout-au-dhow-rabat
14
https://www.ticket.ma/event/convention-nature-4-ever-au-maroc-le-28-septembre-2019-rabat
15
Done


In [38]:
images

['https://www.ticket.ma/upload/events/2051/original_650_4.png',
 'https://www.ticket.ma/upload/events/2050/original_convention_rabat_650X445px_2.png']

## Tout regrouper en une liste

In [49]:
prog = []
for i,j,k,l,m,n,o in zip(titres,dates,heures,images,liens,descs,places):
    #Ajouter le lieu
    t = (i,j,k,l,m,n,o)
    prog.append(t)

In [50]:
prog

[('SQNC.\n\n\t\t\t\t\t\t presents RHADOO',
  'samedi 03 août 2019',
  '16h00',
  'https://www.ticket.ma/upload/events/2051/original_650_4.png',
  'https://www.ticket.ma/event/sqnc-le-03-aout-au-dhow-rabat',
  "Our next guest is Rhadoo, founder of the romanian label\xa0[a:rpia:r]\xa0alongside raresh and petre inspirescu.\n\xa0\nLine up :\n\xa0\n\nrhadoo //\xa0[a:rpia:r]\xa0\r\n\t\xa0\nadam dirk'heim // sqnc.\r\n\t\xa0\njilaa // sqnc.\r\n\t\xa0\nleoma // default settings\r\n\t\xa0\nnathabes // [alt]ernative\n\n\xa0\n*********\r\n\xa0\npresales are the only way to ensure your entrance d-day // les préventes assurent l'entrée le jour-j\n\xa0\nlimited tickets at the door // tickets limités a la porte\n\xa0\n-Prix unique du ticket : 200 mad\n\xa0\n-Lien facebook de l'évènement :\xa0https://www.facebook.com/events/330019124332660/?ti=ia",
  'Le Dhow'),
 ('Convention Natura4Ever au Maroc',
  'samedi 28 septembre 2019',
  '13h30',
  'https://www.ticket.ma/upload/events/2050/original_convention_

### Convertir en objet pandas

In [51]:
import pandas as pd

In [52]:
df = pd.DataFrame(prog,columns=['noms','date','heure','image','lien','description','lieu'])

In [53]:
df

,noms,date,heure,image,lien,description,lieu
0,SQNC.\n\n\t\t\t\t\t\t presents RHADOO,samedi 03 août 2019,16h00,https://www.ticket.ma/upload/events/2051/origi...,https://www.ticket.ma/event/sqnc-le-03-aout-au...,"Our next guest is Rhadoo, founder of the roman...",Le Dhow
1,Convention Natura4Ever au Maroc,samedi 28 septembre 2019,13h30,https://www.ticket.ma/upload/events/2050/origi...,https://www.ticket.ma/event/convention-nature-...,"Vos leaders Natura4Ever vous accueillent, le S...",ZENITH CENTER Rabat


### Exporter en JSON

In [55]:
df.to_json("scraping_json/ticket_records",orient="records")